In [58]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white'})

folder = '../data/experiments'
merge_file = 'pilot_random1'
plot_folder = '../plots/pilot_random1'

experiments = [
    # {
    #     'pagetimes': 'PageTimes-2021-12-09.csv',
    #     'main_part': 'main_part_pilot_2021-12-09.csv',
    #     'folder': f'{folder}/prepilot'
    # },
    # {
    #     'pagetimes': 'PageTimes-2021-12-13.csv',
    #     'main_part': 'main_part_pilot_2021-12-13.csv',
    #     'folder': f'{folder}/prepilot2'
    # }, 
    # {
    #     'pagetimes': 'PageTimes-2021-12-20.csv',
    #     'main_part': 'main_part_pilot_cont_2021-12-20.csv',
    #     'folder': f'{folder}/prepilot4'
    # }, 
    {
        'pagetimes': 'PageTimes-2022-01-13.csv',
        'main_part': 'main_part_pilot_cont_2022-01-13.csv',
        'folder': f'{folder}/prepilot5',
        'episode_length': 8,
    }, 
    {
        'pagetimes': 'PageTimes-2022-01-27.csv',
        'main_part': 'main_part_pilot_cont_2022-01-27.csv',
        'folder': f'{folder}/pilot1',
        'episode_length': 8,
    }, 
    {
        'pagetimes': 'PageTimes-2022-03-21.csv',
        'main_part': 'main_part_pilot_random_2022-03-21.csv',
        'folder': f'{folder}/prepilotRand1',
        'episode_length': 16,
    }, 
]


session_map = {
    # 'sdvfxj22': 'flextimes', 
    # 'n8op39bm': 'fixedtimes',
    # 'qeklctjv': 'continous_exp',
    # 'a4ly7fp4': 'continous_cheap',
    # 'yi3xaim0': 'trail_rounds',
    'sdvfxj22': 'trail_rounds_2',
    'dbcrara4': 'trail_rounds_2',
    'mqqsqhav': 'trail_rounds_2',
    'yaxdn9bp': 'random_1',
    '8l9e0w0q': 'random_1',
    'd81skd2r': 'random_1'
}

exp_pool = {
    'flextimes': 'discrete', 
    'fixedtimes': 'discrete',
    'continous_exp': 'continous',
    'continous_cheap': 'continous',
    'trail_rounds': 'trail_rounds'
}



dfs = []
dfts = []
for exp in experiments:
    df = pd.read_csv(os.path.join(exp['folder'], exp['main_part']))
    dft = pd.read_csv(os.path.join(exp['folder'], exp['pagetimes']))
    print(f"Sessions in folder {exp['folder']} \n{df['session.code'].value_counts()} \n")
    df['experiment_name'] = df['session.code'].map(session_map)
    dft['experiment_name'] = dft['session_code'].map(session_map)
    df['experiment_pool'] = df['experiment_name'].map(exp_pool)
    df['episode_length'] = exp['episode_length']
    dfs.append(df)
    dfts.append(dft)
df = pd.concat(dfs)
dft = pd.concat(dfts)

print(f"Experiments \n{df['experiment_name'].value_counts()} \n")

Sessions in folder ../data/experiments/prepilot5 
yi3xaim0    960
1g4f5wxy    160
xl4vfraf    160
Name: session.code, dtype: int64 

Sessions in folder ../data/experiments/pilot1 
mqqsqhav    3200
dbcrara4    2720
hmjq6293     160
Name: session.code, dtype: int64 

Sessions in folder ../data/experiments/prepilotRand1 
8l9e0w0q    4160
d81skd2r    1600
yaxdn9bp    1280
v13x275n     128
lpc387g3     128
e0z791yu      64
bcypr10f      64
Name: session.code, dtype: int64 

Experiments 
random_1          7040
trail_rounds_2    5920
Name: experiment_name, dtype: int64 



In [59]:
where = (
    (df['player.role'] != 'manager') &
    (~df['player.role'].isnull()) 
)

df = df[where]

assert df.groupby(['session.code', 'subsession.round_number', 'participant.code'])['participant.label'].count().max() == 1

In [60]:
max_player = 4

def get_punishment(row):
    member_idx = row['player.role'][-1]
    return row[f"group.punishment{member_idx}"]

rename = {
    'session.code': 'session', 
    'group.id_in_subsession': 'group_id', 
    'subsession.round_number': 'round_number',
    'participant.code': 'participant_code',
    'player.contribution': 'contribution',
    'player.no_input': 'player_no_input',
    'player.contribution': 'contribution',
    'player.punishment_s': 'punishment_rule_s',
    'player.punishment_b': 'punishment_rule_b',
    'player.punishment_c': 'punishment_rule_c',
    'player.punishment': 'punishment',
    'group.no_input_manager': 'manager_no_input',
    'group.total_contribution': 'total_contribution',
    'group.common_good': 'common_good',
    'group.total_punishment': 'total_punishment',
    'episode': 'episode',
    'experiment_name': 'experiment_name',
    'player.role': 'player_role',
    'player.payoff': 'payoff',
    'experiment_pool': 'experiment_pool',
    'episode_length': 'episode_length',
}

df['episode'] = (df['subsession.round_number'] - 1) // df['episode_length'] + 1
df['player.punishment'] = df.apply(get_punishment, axis=1)

df = df[[k for k in rename.keys() if k in df.columns]]
df = df.rename(columns=rename)

df['round_number'] = (df['round_number']-1) % df['episode_length']
df['global_group_id'] = df['session'] + ' #' + df['group_id'].astype('str')
df['player_id'] = df['player_role'].str.strip().str[-1].astype(int) - 1

In [61]:
# removing unknown sessions (i.e. test sessions)
w_no_exp = df['experiment_name'].isnull()

print(f"{w_no_exp.sum()} removed rows.")
print(f"Removed Sessions \n {df.loc[w_no_exp, 'session'].value_counts()}")

df = df[~w_no_exp]

848 removed rows.
Removed Sessions 
 yi3xaim0    400
1g4f5wxy     64
xl4vfraf     64
hmjq6293     64
e0z791yu     64
bcypr10f     64
v13x275n     64
lpc387g3     64
Name: session, dtype: int64


In [62]:
# removing out participants with no input in any round

w = df.groupby(['global_group_id', 'episode', 'participant_code'])['player_no_input'].transform('all')
print(f"{w.sum()} removed rows.")
df = df[~w]

0 removed rows.


In [63]:
groupby = ['global_group_id', 'round_number', 'episode']

check = (df['punishment'].isnull() == df['contribution'].isnull())
assert check.all(), 'If punishment is null, contribution is null as well.'
n_p_null = df['punishment'].isnull().sum()
print(f"""
There are {n_p_null} rows with null punishments. These are likely from aborted games. We will remove these rows.
""")
df = df[~df['punishment'].isnull()]

# check if punishments are zero if manager has no input
if 'manager_no_input' in df.columns:
    where = df['manager_no_input'] == 1
    check = df[where]['punishment'] == 0
    assert check.all(), 'Inconsistency!'


# check if contributions are zero if player has no input
where = df['player_no_input'] == 1
check = df[where]['contribution'] == 0
assert check.all(), 'Inconsistency!'



check = df.groupby(groupby)['total_contribution'].transform('first') == df.groupby(groupby)['contribution'].transform('sum')
assert check.all(), 'Sum of total contributions does not match group contribution.'


There are 92 rows with null punishments. These are likely from aborted games. We will remove these rows.



We impute missing contributions with 20s and missing punishments with 0s values. This corresponds to a majority imputation.

In [64]:
# for debugging

# df[~check].sort_values(['session','group_id', 'round_number'])

In [65]:
# we might have different number of rows per round, due to some participants dropping out
check = df.groupby(groupby)['total_contribution'].count()
check.value_counts()

4    1461
Name: total_contribution, dtype: int64

In [66]:
# we have different number of round per episode, due to some episodes failing early
check = df.groupby(groupby[:-1])['total_contribution'].count()
check.value_counts()

4    787
8    337
Name: total_contribution, dtype: int64

In [67]:
df['episode_id'] = (df['global_group_id'] + df['episode'].astype(str)).rank(method='dense').astype(int) - 1

In [68]:
df.head()

,session,group_id,round_number,participant_code,contribution,player_no_input,punishment_rule_s,punishment_rule_b,punishment_rule_c,punishment,...,total_punishment,episode,experiment_name,player_role,payoff,experiment_pool,episode_length,global_group_id,player_id,episode_id
160,mqqsqhav,3,0,ac61awuk,15.0,0,NaN,NaN,NaN,0.0,...,0.0,1,trail_rounds_2,group member 1,22.0,NaN,8,mqqsqhav #3,0,112
161,mqqsqhav,9,0,cqtrgylt,5.0,0,NaN,NaN,NaN,0.0,...,0.0,1,trail_rounds_2,group member 2,30.0,NaN,8,mqqsqhav #9,1,124
162,mqqsqhav,2,0,dq7drghw,15.0,0,NaN,NaN,NaN,0.0,...,0.0,1,trail_rounds_2,group member 2,24.0,NaN,8,mqqsqhav #2,1,106
163,mqqsqhav,2,0,u5q0svmp,9.0,0,NaN,NaN,NaN,0.0,...,0.0,1,trail_rounds_2,group member 1,30.0,NaN,8,mqqsqhav #2,0,106
164,mqqsqhav,3,0,k5znx6rn,3.0,0,NaN,NaN,NaN,0.0,...,0.0,1,trail_rounds_2,group member 2,34.0,NaN,8,mqqsqhav #3,1,112


## Final stats

In [69]:
# Total number of groups and participants

df[['global_group_id', 'participant_code']].nunique()

global_group_id      92
participant_code    411
dtype: int64

In [70]:
# Total number of groups and participants per experiment_name

df.groupby(['experiment_name'])[['global_group_id', 'participant_code']].nunique()

,global_group_id,participant_code
experiment_name,,
random_1,49,196
trail_rounds_2,43,215


In [71]:
df.to_csv(os.path.join(folder, f'{merge_file}_player_round.csv'), index=False)

In [72]:
selected_columns = ['session', 'global_group_id','episode', 'episode_id', 'experiment_name', 'round_number', 'participant_code', 'player_no_input', 'manager_no_input', 'player_id', 'contribution', 'punishment', 'payoff', 'common_good']

df[selected_columns].to_csv(os.path.join(folder, f'{merge_file}_player_round_slim.csv'), index=False)